In [ ]:
from vitessce import VitessceConfig, 
from vitessce import CoordinationTypes as ct

In [ ]:
# Use Case 1: Visualize 3 pairs of scatterplots
# A mock up of a potential API pattern:

vc = VitessceConfig() # Also allow to pass in a JSON-based view config to modify it (e.g. if coming from the portal)

## Add data.
### if dataset contained in local AnnData objects:
disease_dataset = vc.add_dataset(data=lung_covid_anndata)
healthy_dataset = vc.add_dataset(data=lung_healthy_anndata)
### or, if dataset is located in remote files:
disease_dataset = vc.add_dataset(name="COVID-19 patient PBMCs")
healthy_dataset = vc
    .add_dataset(name="Healthy patient PBMCs")
    .add_file_url("http://s3.com/disease-cells.json", data_type='cells', file_type='cells.json')
healthy_dataset
    .add_file_url("http://s3.com/healthy-cells.json", data_type='cells', file_type='cells.json')

## Add views.
scatterplot_disease_pca = vc.add_view(disease_dataset, "scatterplot", mapping="X_pca")
scatterplot_disease_umap = vc.add_view(disease_dataset, "scatterplot", mapping="X_umap")
scatterplot_disease_tsne = vc.add_view(disease_dataset, "scatterplot", mapping="X_tsne")
scatterplot_healthy_pca = vc.add_view(healthy_dataset, "scatterplot", mapping="X_pca")
scatterplot_healthy_umap = vc.add_view(healthy_dataset, "scatterplot", mapping="X_umap")
scatterplot_healthy_tsne = vc.add_view(healthy_dataset, "spatial", mapping="X_tsne")
    .add_spatial_layer(...c)

## Link views. Use enums for coordination types to enable type checking.
vc.link_views(scatterplot_disease_pca, scatterplot_healthy_pca, coordination_types=[ct.EMBEDDING_ZOOM, ct.EMBEDDING_TARGET_X, ct.EMBEDDING_TARGET_Y])
vc.link_views(scatterplot_disease_umap, scatterplot_healthy_umap, coordination_types=[ct.EMBEDDING_ZOOM, ct.EMBEDDING_TARGET_X, ct.EMBEDDING_TARGET_Y])
vc.link_views(scatterplot_disease_tsne, scatterplot_healthy_tsne, coordination_types=[ct.EMBEDDING_ZOOM, ct.EMBEDDING_TARGET_X, ct.EMBEDDING_TARGET_Y])

### or, a more Altair-like API for linking views:
ez1 = vc.add_coordination(ct.EMBEDDING_ZOOM) # obtain a reference to a new coordination scope
ez2 = vc.add_coordination(ct.EMBEDDING_ZOOM)
ez3 = vc.add_coordination(ct.EMBEDDING_ZOOM)

etx1 = vc.add_coordination(ct.EMBEDDING_TARGET_X)
etx2 = vc.add_coordination(ct.EMBEDDING_TARGET_X)
etx3 = vc.add_coordination(ct.EMBEDDING_TARGET_X)

ety1 = vc.add_coordination(ct.EMBEDDING_TARGET_Y)
ety2 = vc.add_coordination(ct.EMBEDDING_TARGET_Y)
ety3 = vc.add_coordination(ct.EMBEDDING_TARGET_Y)

scatterplot_disease_pca
    .use_coordination(ez1) # use each coordination scope by passing its reference
    .use_coordination(etx1)
    .use_coordination(ety1)
scatterplot_healthy_pca
    .use_coordination(ez1)
    .use_coordination(etx1)
    .use_coordination(ety1)

#### or, could allow multiple scopes to be passed at once:
scatterplot_disease_umap
    .use_coordination(ez2, etx2, ety2)
scatterplot_healthy_umap
    .use_coordination(ez2, etx2, ety2)
    
scatterplot_disease_tsne
    .use_coordination(ez3, etx3, ety3)
scatterplot_healthy_tsne
    .use_coordination(ez3, etx3, ety3)

## Set the layout. (Should also be optional - try to set up automatically if not specified.)
vc.layout = ( # Dont worry about magic syntax in first iteration.
    (scatterplot_disease_pca / scatterplot_healthy_pca)
    | (scatterplot_disease_umap / scatterplot_healthy_umap)
    | (scatterplot_disease_tsne / scatterplot_healthy_tsne)
)
### or
vc.layout = hconcat(
    vconcat(scatterplot_disease_pca, scatterplot_healthy_pca),
    vconcat(scatterplot_disease_umap, scatterplot_healthy_umap),
    vconcat(scatterplot_disease_tsne, scatterplot_healthy_tsne)
)
### or, when adding views, set x,y,w,h parameters directly rather than the separate vc.layout step:
scatterplot1 = vc.add_view(dataset1, "scatterplot", mapping="X_pca", x=0, y=0, w=4, h=4)
scatterplot2 = vc.add_view(dataset2, "scatterplot", mapping="X_umap", x=4, y=0, w=4, h=4)

# Think about method chaining. For example, add file URL should return the dataset to allow chaining.